## Projet 9 : Produisez une étude de marché avec R ou Python


### Mission: 

> Pour le lancement à l’international de l'entreprise agroalimentaire française La Poule qui Chante, proposer une première analyse des groupements de pays que l’on peut cibler pour exporter nos poulets. 

## Notebook 1 : Préparation, Nettoyage des Données et Analyse Exploratoire

> Dans ce notebook, nous allons importer les données de ventes de la librairie et nettoyer les données pour nous assurer qu'elles sont prêtes à être analysées. Ensuite nous ferons une première analyse exploratoire des données.

## Sommaire :


### Partie 1: Préparation, Nettoyage des Données et Analyse Exploratoire (Notebook 1)

### [1) Importation des données](#A1)
### [2) Nettoyage et préparation des données](#A2)

- [1.Table Disponibilite](#A2.1)
- [2.Table Population](#A2.2)
- [3.Table Demographie](#A2.3)
- [4.Table Stabilite](#A2.4)
- [5.Table Economie](#A2.5)
 
### Partie 2: Analyse des Clusterings et  visualisations (Notebook 2)

#### [1. La Méthode de Classification Ascendante Hiérarchique (CAH) avec le Dendrogramme de Visualisation](#B1)


#### [2. L’analyse par la Méthode des k-means](#B2)

#### [3. Comparaison des Résultats des deux Méthodes de Clustering](#B3)

#### [4. Analyse des Centroïdes des Classes](#B4)

#### [5. Réaliser une ACP pour comprendre les groupes, les liens entre les variables et les liens entre les pays](#B5)


## Partie 1: Importation, nettoyage et préparation des données

### <a id="A1">1) Importation des données</a>


In [1]:
# Import des librairies et fonctions nécessaires aux analyses

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from scipy import stats
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import silhouette_score

import warnings
warnings.simplefilter("ignore")

In [2]:
# Formatage du Notebook 

# Création d'une classe nommée 'color' pour mettre en forme nos résultats.
class color:
   BLUE = '\033[94m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [3]:
# Chargement des données disponibilite_2017 et population_2000_2018
disponibilite = pd.read_csv('DisponibiliteAlimentaire_2017.csv')
population_2000_2018 = pd.read_csv('population_2000_2018.csv')
demographie = pd.read_csv('Démographie.csv')
stabilite = pd.read_csv('Stabilite_Politique.csv')
economie = pd.read_csv('croissance_eco.csv')

### <a id="A2">2) Nettoyage et préparation des données</a>

### <a id="A2.1">1.Table Disponibilite</a>

In [4]:
# Afficher les premières lignes des données Disponibilite_2017

disponibilite = pd.read_csv('DisponibiliteAlimentaire_2017.csv')
disponibilite.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2511,Blé et produits,2017,2017,Milliers de tonnes,4281.0,S,Données standardisées
1,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,2302.0,S,Données standardisées
2,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2511,Blé et produits,2017,2017,Milliers de tonnes,-119.0,S,Données standardisées
3,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5911,Exportations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,0.0,S,Données standardisées
4,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2511,Blé et produits,2017,2017,Milliers de tonnes,6701.0,S,Données standardisées


In [5]:
# Filtrer les données pour la viande de volailles (Poulet)

disponibilite = disponibilite.loc[disponibilite['Produit'] == 'Viande de Volailles']
disponibilite.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
651,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2734,Viande de Volailles,2017,2017,Milliers de tonnes,28.0,S,Données standardisées
652,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2734,Viande de Volailles,2017,2017,Milliers de tonnes,29.0,S,Données standardisées
653,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2734,Viande de Volailles,2017,2017,Milliers de tonnes,0.0,S,Données standardisées
654,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2734,Viande de Volailles,2017,2017,Milliers de tonnes,57.0,S,Données standardisées
655,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5123,Pertes,2734,Viande de Volailles,2017,2017,Milliers de tonnes,2.0,S,Données standardisées


In [6]:
# Pivoter la colonne 'Élément'

disponibilite = disponibilite.pivot_table(index=['Zone', 'Année', 'Produit'],
                                                         columns='Élément', values='Valeur').reset_index()
disponibilite.head()

Élément,Zone,Année,Produit,Alimentation pour touristes,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Résidus,Semences,Traitement,Variation de stock
0,Afghanistan,2017,Viande de Volailles,NaN,NaN,NaN,5.0,1.53,0.33,0.54,57.0,NaN,29.0,55.0,2.0,28.0,0.0,NaN,NaN,0.0
1,Afrique du Sud,2017,Viande de Volailles,0.0,NaN,NaN,143.0,35.69,9.25,14.11,2118.0,63.0,514.0,2035.0,83.0,1667.0,0.0,NaN,NaN,0.0
2,Albanie,2017,Viande de Volailles,NaN,NaN,NaN,85.0,16.36,6.45,6.26,47.0,0.0,38.0,47.0,NaN,13.0,0.0,NaN,NaN,4.0
3,Algérie,2017,Viande de Volailles,0.0,NaN,NaN,22.0,6.38,1.50,1.97,277.0,0.0,2.0,264.0,13.0,275.0,0.0,NaN,NaN,0.0
4,Allemagne,2017,Viande de Volailles,NaN,NaN,NaN,71.0,19.47,4.16,7.96,1739.0,646.0,842.0,1609.0,NaN,1514.0,-38.0,NaN,167.0,-29.0


In [7]:
# Selectionner les données utiles

disponibilite = disponibilite[['Zone', 'Disponibilité alimentaire en quantité (kg/personne/an)', 'Production', 
                               'Importations - Quantité', 'Nourriture']]

# Renommer les variable

disponibilite.rename(columns = {'Disponibilité alimentaire en quantité (kg/personne/an)': 'dispo_alim_kg/pers/an',
                                 'Importations - Quantité': 'Importations'},inplace=True)
disponibilite.columns

Index(['Zone', 'dispo_alim_kg/pers/an', 'Production', 'Importations',
       'Nourriture'],
      dtype='object', name='Élément')

In [8]:
# Vérifier les informations sur les données

disponibilite.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Zone                   172 non-null    object 
 1   dispo_alim_kg/pers/an  172 non-null    float64
 2   Production             168 non-null    float64
 3   Importations           170 non-null    float64
 4   Nourriture             170 non-null    float64
dtypes: float64(4), object(1)
memory usage: 6.8+ KB


In [9]:
# Vérifier les doublons 

doublons = disponibilite.duplicated(subset='Zone')
if doublons.any():
    print("Il y a des doublons dans la table disponibilite.")
else:
    print("Il n'y a pas de doublons dans la table disponibilite.")

Il n'y a pas de doublons dans la table disponibilite.


In [10]:
# Vérification des valeurs aberrantes et atypiques

disponibilite.describe()

Élément,dispo_alim_kg/pers/an,Production,Importations,Nourriture
count,172.000000,168.000000,170.000000,170.000000
mean,20.213372,725.190476,89.529412,657.047059
std,15.860311,2501.457125,186.669983,2136.545796
min,0.130000,0.000000,0.000000,2.000000
25%,6.440000,13.750000,3.000000,28.500000
50%,18.090000,70.000000,16.000000,99.500000
75%,30.037500,409.750000,81.250000,365.250000
max,72.310000,21914.000000,1069.000000,18100.000000


### <a id="A2.2">2.Table Population</a>

In [11]:
# Afficher les premières lignes des données Population_2000_2018

population = pd.read_csv('population_2000_2018.csv')
population.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2000,2000,1000 personnes,20779.953,X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2001,2001,1000 personnes,21606.988,X,Sources internationales sûres,NaN
2,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2002,2002,1000 personnes,22600.770,X,Sources internationales sûres,NaN
3,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2003,2003,1000 personnes,23680.871,X,Sources internationales sûres,NaN
4,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2004,2004,1000 personnes,24726.684,X,Sources internationales sûres,NaN


In [12]:
# Filtrer les données de population sur l'année 2017

population = population_2000_2018.loc[population_2000_2018['Année'] == 2017].reset_index()
population.head()

,index,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,17,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,36296.113,X,Sources internationales sûres,NaN
1,36,OA,Séries temporelles annuelles,202,Afrique du Sud,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,57009.756,X,Sources internationales sûres,NaN
2,55,OA,Séries temporelles annuelles,3,Albanie,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,2884.169,X,Sources internationales sûres,NaN
3,74,OA,Séries temporelles annuelles,4,Algérie,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,41389.189,X,Sources internationales sûres,NaN
4,93,OA,Séries temporelles annuelles,79,Allemagne,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,82658.409,X,Sources internationales sûres,NaN


In [13]:
# Sélectionner les variables pertinentes 

population = population[['Zone', 'Valeur']]

# Renommer la variable 'Valeur' en 'Population'

population.rename(columns = {'Valeur': 'Population'}, inplace = True)
population.head()

,Zone,Population
0,Afghanistan,36296.113
1,Afrique du Sud,57009.756
2,Albanie,2884.169
3,Algérie,41389.189
4,Allemagne,82658.409


In [14]:
# Colonne Population: changement d'unité '1000 personnes' en 'personnes'

population['Population'] = population['Population'] * 1000
population.head()

,Zone,Population
0,Afghanistan,36296113.0
1,Afrique du Sud,57009756.0
2,Albanie,2884169.0
3,Algérie,41389189.0
4,Allemagne,82658409.0


In [15]:
# Vérifier les informations sur les données

population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Zone        236 non-null    object 
 1   Population  236 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.8+ KB


In [16]:
# Vérification des doublons

population.duplicated().sum()

0

In [17]:
# Changement du type de la variable 'Population'

population['Population'] = population['Population'].astype(int)
population['Population'].dtypes

dtype('int32')

In [18]:
# Vérification des valeurs aberrantes et atypiques

population.describe()

,Population
count,2.360000e+02
mean,3.198362e+07
std,1.318949e+08
min,7.930000e+02
25%,3.803032e+05
50%,5.203510e+06
75%,1.930842e+07
max,1.421022e+09


### <a id="A2.3">3.Table Demographie</a>

In [19]:
# Affichage des données de Démographie

demographie = pd.read_csv('Démographie.csv')
demographie.head()

,Code Domaine,Domaine,Code zone (M49),Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,4,Afghanistan,551,Population rurale,3010,Population-Estimations,2017,2017,1000 personnes,26558.609,X,Ciffre de sources internationales,NaN
1,OA,Séries temporelles annuelles,4,Afghanistan,561,Population urbaine,3010,Population-Estimations,2017,2017,1000 personnes,8971.472,X,Ciffre de sources internationales,NaN
2,OA,Séries temporelles annuelles,710,Afrique du Sud,551,Population rurale,3010,Population-Estimations,2017,2017,1000 personnes,19369.002,X,Ciffre de sources internationales,NaN
3,OA,Séries temporelles annuelles,710,Afrique du Sud,561,Population urbaine,3010,Population-Estimations,2017,2017,1000 personnes,37348.154,X,Ciffre de sources internationales,NaN
4,OA,Séries temporelles annuelles,8,Albanie,551,Population rurale,3010,Population-Estimations,2017,2017,1000 personnes,1190.155,X,Ciffre de sources internationales,NaN


In [20]:
# Sélectionner les variables pertinentes 

demographie = demographie[['Zone','Élément','Valeur']]

# Calcul du nombre de personnes sur la Variable 'Valeur' en multipliant par 1000

demographie['Valeur'] = demographie['Valeur'] * 1000
demographie.head()

,Zone,Élément,Valeur
0,Afghanistan,Population rurale,26558609.0
1,Afghanistan,Population urbaine,8971472.0
2,Afrique du Sud,Population rurale,19369002.0
3,Afrique du Sud,Population urbaine,37348154.0
4,Albanie,Population rurale,1190155.0


In [21]:
# Pivoter la colonne 'Élément' en colonnes avec les valeurs correspondantes dans 'Valeur'

demographie = demographie.pivot(index='Zone', columns='Élément', values='Valeur').reset_index()
demographie.head()

Élément,Zone,Population rurale,Population urbaine
0,Afghanistan,26558609.0,8971472.0
1,Afrique du Sud,19369002.0,37348154.0
2,Albanie,1190155.0,1740032.0
3,Algérie,11547532.0,29770610.0
4,Allemagne,18672100.0,63442124.0


In [22]:
# Vérifier les informations sur les données

demographie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Zone                234 non-null    object 
 1   Population rurale   234 non-null    float64
 2   Population urbaine  234 non-null    float64
dtypes: float64(2), object(1)
memory usage: 5.6+ KB


In [23]:
# Normalisation des types

demographie['Population rurale'] = demographie['Population rurale'].astype(int)
demographie['Population urbaine'] = demographie['Population urbaine'].astype(int)
demographie.dtypes

Élément
Zone                  object
Population rurale      int32
Population urbaine     int32
dtype: object

In [24]:
# Vérifier les doublons

demographie.duplicated().sum()

0

In [25]:
# Vérifier des valeurs aberrantes et atypiques

demographie.describe()

Élément,Population rurale,Population urbaine
count,2.340000e+02,2.340000e+02
mean,1.712774e+07,2.129702e+07
std,8.085946e+07,8.544778e+07
min,0.000000e+00,0.000000e+00
25%,8.656050e+04,3.163762e+05
50%,1.567076e+06,3.129546e+06
75%,9.470357e+06,1.011654e+07
max,8.892167e+08,8.433148e+08


### <a id="A2.4">4.Table Stabilite</a>

In [26]:
# Afficher les données

stabilite = pd.read_csv('Stabilite_Politique.csv')
stabilite.head()

,Code Domaine,Domaine,Code zone (M49),Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,FS,Données de la sécurité alimentaire,4,Afghanistan,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2017,2017,indice,-2.80,X,Ciffre de sources internationales,NaN
1,FS,Données de la sécurité alimentaire,710,Afrique du Sud,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2017,2017,indice,-0.28,X,Ciffre de sources internationales,NaN
2,FS,Données de la sécurité alimentaire,8,Albanie,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2017,2017,indice,0.38,X,Ciffre de sources internationales,NaN
3,FS,Données de la sécurité alimentaire,12,Algérie,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2017,2017,indice,-0.92,X,Ciffre de sources internationales,NaN
4,FS,Données de la sécurité alimentaire,276,Allemagne,6125,Valeur,21032,Stabilité politique et absence de violence/ter...,2017,2017,indice,0.59,X,Ciffre de sources internationales,NaN


In [27]:
# Sélectionner les variables pertinentes 

stabilite = stabilite[['Zone', 'Produit', 'Valeur']]

# Pivoter la colonne 'Produit' 
stabilite = stabilite.pivot(index='Zone', columns='Produit', values='Valeur').reset_index()
stabilite.head()

Produit,Zone,Stabilité politique et absence de violence/terrorisme (indice)
0,Afghanistan,-2.80
1,Afrique du Sud,-0.28
2,Albanie,0.38
3,Algérie,-0.92
4,Allemagne,0.59


In [28]:
# Renommer la variable 

stabilite.rename(columns ={'Stabilité politique et absence de violence/terrorisme (indice)': 'Indice de Stabilité Politique'}, inplace = True)
stabilite.columns

Index(['Zone', 'Indice de Stabilité Politique'], dtype='object', name='Produit')

In [29]:
# Vérifier les informations sur les données

stabilite.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Zone                           197 non-null    object 
 1   Indice de Stabilité Politique  197 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.2+ KB


In [30]:
# Vérifier les doublons

stabilite.duplicated().sum()

0

### <a id="A2.5">5. Table Economie</a>

In [31]:
# Afficher les données

economie = pd.read_csv('croissance_eco.csv')
economie.head()

,Code Domaine,Domaine,Code zone (M49),Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,MK,Indicateurs macro,4,Afghanistan,6119,Valeur US $ par habitant,22008,Produit Intérieur Brut,2017,2017,US$,530.149831,X,Ciffre de sources internationales,NaN
1,MK,Indicateurs macro,4,Afghanistan,61290,Croissance annuelle US$ par habitant,22008,Produit Intérieur Brut,2017,2017,%,1.902494,X,Ciffre de sources internationales,NaN
2,MK,Indicateurs macro,710,Afrique du Sud,6119,Valeur US $ par habitant,22008,Produit Intérieur Brut,2017,2017,US$,6723.928582,X,Ciffre de sources internationales,NaN
3,MK,Indicateurs macro,710,Afrique du Sud,61290,Croissance annuelle US$ par habitant,22008,Produit Intérieur Brut,2017,2017,%,17.248840,X,Ciffre de sources internationales,NaN
4,MK,Indicateurs macro,8,Albanie,6119,Valeur US $ par habitant,22008,Produit Intérieur Brut,2017,2017,US$,4521.752219,X,Ciffre de sources internationales,NaN


In [32]:
# Pivoter la colonne 'Element' et Sélectionner les variables pertinentes
economie = economie.pivot_table(index=['Zone'], 
                                columns='Élément',
                                values='Valeur').reset_index()
economie.head()

Élément,Zone,Croissance annuelle US$ par habitant,Valeur US $ par habitant
0,Afghanistan,1.902494,530.149831
1,Afrique du Sud,17.248840,6723.928582
2,Albanie,9.831056,4521.752219
3,Algérie,4.228057,4134.936054
4,Allemagne,6.063881,44670.222288


In [33]:
# Renommer les variables

economie.rename(columns = {'Valeur US $ par habitant': 'PIB par Habitant', 
                           'Croissance annuelle US$ par habitant': 'Taux de croissance'}, inplace = True)
economie.columns

Index(['Zone', 'Taux de croissance', 'PIB par Habitant'], dtype='object', name='Élément')

In [34]:
# Vérifier les informations sur les données

economie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Zone                211 non-null    object 
 1   Taux de croissance  211 non-null    float64
 2   PIB par Habitant    211 non-null    float64
dtypes: float64(2), object(1)
memory usage: 5.1+ KB


In [35]:
# Arrondir au décimal

economie['Taux de croissance'] = round(economie['Taux de croissance'], 2)
economie['PIB par Habitant'] = round(economie['PIB par Habitant'], 2)
economie.head()

Élément,Zone,Taux de croissance,PIB par Habitant
0,Afghanistan,1.90,530.15
1,Afrique du Sud,17.25,6723.93
2,Albanie,9.83,4521.75
3,Algérie,4.23,4134.94
4,Allemagne,6.06,44670.22


In [36]:
# Vérifier les doublons

economie.duplicated().sum()

0

In [37]:
# Vérifier des valeurs aberrantes et atypiques

economie.describe()

Élément,Taux de croissance,PIB par Habitant
count,211.000000,211.000000
mean,5.865545,16886.035782
std,7.472346,25550.733493
min,-29.220000,295.620000
25%,3.385000,2082.325000
50%,6.000000,6450.320000
75%,8.920000,19806.130000
max,32.520000,173611.810000


- ###  Création de la Base de Données

In [38]:
# Fusionner les Tables en utilisant la clé de jointure 'Zone'

data_etude = disponibilite.merge(population, on='Zone') \
                                .merge(demographie, on='Zone') \
                                .merge(stabilite, on='Zone') \
                                .merge(economie, on='Zone')

# Afficher les premières lignes de la base de données fusionnée
data_etude.head()

,Zone,dispo_alim_kg/pers/an,Production,Importations,Nourriture,Population,Population rurale,Population urbaine,Indice de Stabilité Politique,Taux de croissance,PIB par Habitant
0,Afghanistan,1.53,28.0,29.0,55.0,36296113,26558609,8971472,-2.80,1.90,530.15
1,Afrique du Sud,35.69,1667.0,514.0,2035.0,57009756,19369002,37348154,-0.28,17.25,6723.93
2,Albanie,16.36,13.0,38.0,47.0,2884169,1190155,1740032,0.38,9.83,4521.75
3,Algérie,6.38,275.0,2.0,264.0,41389189,11547532,29770610,-0.92,4.23,4134.94
4,Allemagne,19.47,1514.0,842.0,1609.0,82658409,18672100,63442124,0.59,6.06,44670.22


> Après la jointure interne, le nombre de pays a diminué pour ne conserver que les pays dont les données sont plus complètes.

In [39]:
# Afficher les lignes avec les valeurs manquantes 

data_etude[data_etude.isnull().any(axis=1)]

,Zone,dispo_alim_kg/pers/an,Production,Importations,Nourriture,Population,Population rurale,Population urbaine,Indice de Stabilité Politique,Taux de croissance,PIB par Habitant
41,Djibouti,2.68,NaN,3.0,3.0,944099,213906,743079,-0.74,3.94,2659.85
90,Maldives,13.50,NaN,12.0,7.0,496402,264503,171827,0.23,4.38,10063.00
110,Ouzbékistan,1.96,NaN,NaN,NaN,31959785,15779684,16130957,-0.24,-29.04,1943.34
125,République démocratique populaire lao,10.91,NaN,NaN,NaN,6953035,4501174,2356986,0.40,6.36,2439.46


In [40]:
# Supprimer les lignes avec les valeurs manquantes et réinitialiser l'index

data_etude.dropna(inplace=True)
data_etude.reset_index()

# Vérification
data_etude[data_etude.isnull().any(axis=1)].sum()

Zone                             0.0
dispo_alim_kg/pers/an            0.0
Production                       0.0
Importations                     0.0
Nourriture                       0.0
Population                       0.0
Population rurale                0.0
Population urbaine               0.0
Indice de Stabilité Politique    0.0
Taux de croissance               0.0
PIB par Habitant                 0.0
dtype: float64

> NB: Pour éviter d'avoir un groupe formé d'un seul pays, il est nécessaire de supprimer l'inde de nos analyses pour une meilleure classification des données.

In [41]:
# Suppression de l'Inde
data_etude = data_etude[data_etude['Zone'] != 'Inde']
data_etude.shape[0]

162

### Définition des variables:

> **Zone** : Variable d'identification pour chaque **pays**.

> **Disponibilité alimentaire en quantité (kg/personne/an)** : La quantité de nourriture disponible en kilogrammes par personne et par an dans le pays. Cette variable est utile pour **déterminer les pays où la consommation de poulet est la plus élevée** et donc les pays où il pourrait y avoir une **forte demande pour l'importation de poulet**.

> **Production** : La quantité de viande de volailles produite dans chaque pays, en milliers de tonnes. Cette variable est utile pour **déterminer les pays qui ont une production élevée** et qui pourraient être des **concurrents potentiels** dans le marché de l'exportation de poulet.

> **Importations** : La quantité de viande de volailles importée dans le pays, en milliers de tonnes.  Cette variable indique les niveaux d'importation de poulet dans un pays donné, ce qui **peut indiquer une demande insatisfaite** pour les produits de volailles locaux.

> **Nourriture** : La quantité de viande de volailles consommée dans chaque pays, en milliers de tonnes. Cette variable est utile pour **identifier les pays où la consommation de poulet est la plus élevée** et donc les pays où il pourrait y avoir une forte demande pour l'importation de poulet.

> **Population** : Le nombre total de personnes vivant dans un pays. Cette variable fournit une **indication de la taille du marché potentiel** pour les produits de volaille dans un pays donné.

> **Population rurale** : Le nombre de personnes vivant dans des zones rurales d'un pays. Cette variable peut être utile pour identifier les pays où la production de poulet est peut-être plus courante et donc où **il pourrait y avoir une offre plus importante de poulet**.

> **Population urbaine** : Le nombre de personnes vivant dans des zones urbaines d'un pays. Cette variable peut être utile pour identifier les pays où la **consommation de poulet est peut-être plus courante** et donc où il pourrait y avoir une forte demande pour l'importation de poulet.

> **PIB par Habitant**: Produit Intérieur Brut par habitant dans un pays au cours d'une année, exprimée en dollars américains. Cette variable est utile pour **identifier les pays où les consommateurs ont un pouvoir d'achat élevé** et donc où il pourrait y avoir une **demande plus importante pour le poulet importé**.

> **Taux de croissance** : Le taux de croissance du PIB de chaque pays. Cette variable est utile pour **identifier les pays où l'économie est en croissance** et permet de **déterminer si la demande pour les produits de volailles augmentera ou diminuera à l'avenir**.

> **Indice de stabilité politique** : Un indice qui mesure la stabilité politique. Cette variable est utile pour **identifier les pays où le risque politique est faible**, ce qui peut être un facteur **important dans la prise de décision** d'exporter du poulet. Car un **risque politique faible peut garantir un environnement commercial plus stable et plus sûr**.